In [1]:
import json
import folium
import pandas as pd
import geopandas as gpd
from folium.features import GeoJsonTooltip

from preprocessing_func import * 

In [2]:
points_gdf = process_places("data/CB_points_of_interest.gpkg")
polygons_gdf = process_zones("data/CB_IMD_2019.geojson")
choropleth_gdf = create_choropleth_data(points_gdf, polygons_gdf)

c:\Users\us\anaconda3\envs\detectree2-env\lib\site-packages\IPython\core\interactiveshell.py:3548: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
data_class = 'bar'
datasets = filter_datasets(points_gdf, choropleth_gdf)
markers_gdf = datasets[f"{data_class}_gdf"]
map_gdf = datasets[f"{data_class}_choropleth_gdf"]

In [17]:
# Create map
m = folium.Map(location=[52.205320488806095, 0.11825440531707056], zoom_start=15)

In [18]:
# Create choropleth
choropleth = folium.Choropleth(
    geo_data=map_gdf,
    name=f'Choropleth ({data_class})',
    data=map_gdf,
    columns=['lsoa11nmw', 'count'],
    key_on='feature.properties.lsoa11nmw',
    fill_color='YlGnBu',
    fill_opacity=1,
    line_opacity=0.5,
    legend_name='Counts'
)

# Add tooltip to the choropleth
choropleth.geojson.add_child(
    GeoJsonTooltip(
        fields=['classname', 'lsoa11nmw', 'count'],
        aliases=['Type: ', 'Area: ', 'Count: '],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    )
)

# Add choropleth to the map
choropleth.add_to(m)

In [19]:
# Feature group for points
points_feature_group = folium.FeatureGroup(name=f'Markers ({data_class})').add_to(m)

# Add points to the feature group
for feature in json.loads(markers_gdf.to_json())['features']:
    classname = feature['properties']['classname']
    name = feature['properties']['name']
    address = feature['properties']['street_name']
    url = feature['properties']['url']
    url = '' if url is None else url
    geom = feature['geometry']['coordinates']
    
    marker = folium.Marker(
        location=[geom[1], geom[0]],
        popup=folium.Popup(html=create_popup_html(name, address, url), max_width=300),
        tooltip=folium.Tooltip(text=create_tooltip_html(name)),
        icon=folium.Icon(color='orange', icon='coffee', prefix='fa') if classname == 'Cafes, Snack Bars and Tea Rooms' else folium.Icon(color='darkblue', icon='glass')
    )
    marker.add_to(points_feature_group)

In [20]:
# Add Layer Control to toggle the layers
folium.LayerControl(collapsed=False).add_to(m)

# Save and show map
# m.save('folium_map_with_layer_control.html')
m  # Display the map if using Jupyter Notebooks